# Reading raw data from schema-less format (e.g., csv)

In [56]:
import pandas as pd
import pyarrow as pa

print(f"pandas version: {pd.__version__}")
print(f"pyarrow version: {pa.__version__}")

pandas version: 2.1.2
pyarrow version: 14.0.0


## Example data
As the example data, I will be using the famous NYC taxi data set. I picked it because:
- it contains a good mix of column datatypes that we would want to parse for typical use cases (in particular, including datetime).
- it is large enough (~1.5 million rows) to allow us to do meaningful performance comparisons.

It does not contain any missing values; therefore, I artificially introduced missing values in different ways, so we can look at how this affects Arrow's ability to correctly infer the schema or parse the data into a given schema. Likewise, it will show us whether we get meaningful error messages. I have moved the set up code into a different notebook at data/data_setup.ipynb.

In [53]:
PATH_MISSING_AS_EMPTY_STRING = 'data/missing_as_empty_string.csv'
PATH_MISSING_AS_STRING = 'data/missing_as_string.csv'
PATH_MISSING_AS_MULTIPLE_STRINGS = 'data/missing_as_multiple_strings.csv'
MISSING_AS_QUESTION_MARKS = 'data/missing_as_question_marks.csv'

## 1) Reading using schema *inference*
Note that automatic type inference works for some representations of missing values. Both empty string and "NA" is properly recognized, whereas "???" is not and leads to all columns being interpreted as strings.

### Missing values encoded as EMPTY STRING
Let's take a look at what the data looks like:

In [ ]:
!head -n 4 data/missing_as_empty_string.csv


id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.98215484619139,40.76793670654297,-73.96463012695312,40.765602111816406,N,455.0
,,,,,,,,,,
id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.97902679443358,40.763938903808594,-74.00533294677734,40.710086822509766,N,2124.0


In [5]:
%%time
pd.read_csv(
    PATH_MISSING_AS_EMPTY_STRING,
    engine='pyarrow',
    dtype_backend='pyarrow',
) \
.dtypes

CPU times: user 900 ms, sys: 369 ms, total: 1.27 s
Wall time: 155 ms


id                          string[pyarrow]
vendor_id                   double[pyarrow]
pickup_datetime       timestamp[s][pyarrow]
dropoff_datetime      timestamp[s][pyarrow]
passenger_count             double[pyarrow]
pickup_longitude            double[pyarrow]
pickup_latitude             double[pyarrow]
dropoff_longitude           double[pyarrow]
dropoff_latitude            double[pyarrow]
store_and_fwd_flag          string[pyarrow]
trip_duration               double[pyarrow]
dtype: object

### Missing encoded as 'NA'

In [ ]:
!head -n 4 data/missing_as_string.csv

id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.98215484619139,40.76793670654297,-73.96463012695312,40.765602111816406,N,455.0
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.97902679443358,40.763938903808594,-74.00533294677734,40.710086822509766,N,2124.0


In [33]:
%%time
df = pd.read_csv(
    PATH_MISSING_AS_STRING,
    engine='pyarrow',
    dtype_backend='pyarrow',
)

df.dtypes

CPU times: user 966 ms, sys: 325 ms, total: 1.29 s
Wall time: 199 ms


id                          string[pyarrow]
vendor_id                   double[pyarrow]
pickup_datetime       timestamp[s][pyarrow]
dropoff_datetime      timestamp[s][pyarrow]
passenger_count             double[pyarrow]
pickup_longitude            double[pyarrow]
pickup_latitude             double[pyarrow]
dropoff_longitude           double[pyarrow]
dropoff_latitude            double[pyarrow]
store_and_fwd_flag          string[pyarrow]
trip_duration               double[pyarrow]
dtype: object

In [29]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2.0,2016-03-14 17:24:55,2016-03-14 17:32:30,1.0,-73.982155,40.767937,-73.96463,40.765602,N,455.0
1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,id3858529,2.0,2016-01-19 11:35:24,2016-01-19 12:10:48,1.0,-73.979027,40.763939,-74.005333,40.710087,N,2124.0
3,id3504673,2.0,2016-04-06 19:32:31,2016-04-06 19:39:40,1.0,-74.01004,40.719971,-74.012268,40.706718,N,429.0
4,id2181028,2.0,2016-03-26 13:30:55,2016-03-26 13:38:10,1.0,-73.973053,40.793209,-73.972923,40.78252,N,435.0


### Missing encoded as 'NA' or 'nan'

In [59]:
%%time
df = pd.read_csv(
    PATH_MISSING_AS_MULTIPLE_STRINGS,
    engine='pyarrow',
    dtype_backend='pyarrow',
)

df.dtypes

CPU times: user 860 ms, sys: 1.13 s, total: 1.99 s
Wall time: 130 ms


id                          string[pyarrow]
vendor_id                   double[pyarrow]
pickup_datetime       timestamp[s][pyarrow]
dropoff_datetime      timestamp[s][pyarrow]
passenger_count             double[pyarrow]
pickup_longitude            double[pyarrow]
pickup_latitude             double[pyarrow]
dropoff_longitude           double[pyarrow]
dropoff_latitude            double[pyarrow]
store_and_fwd_flag          string[pyarrow]
trip_duration               double[pyarrow]
dtype: object

### Missing values encoded as "???"

In [30]:
%%time
pd.read_csv(
    MISSING_AS_QUESTION_MARKS,
    engine='pyarrow',
    dtype_backend='pyarrow',
) \
.dtypes

CPU times: user 1.03 s, sys: 1.34 s, total: 2.37 s
Wall time: 312 ms


id                    string[pyarrow]
vendor_id             string[pyarrow]
pickup_datetime       string[pyarrow]
dropoff_datetime      string[pyarrow]
passenger_count       string[pyarrow]
pickup_longitude      string[pyarrow]
pickup_latitude       string[pyarrow]
dropoff_longitude     string[pyarrow]
dropoff_latitude      string[pyarrow]
store_and_fwd_flag    string[pyarrow]
trip_duration         string[pyarrow]
dtype: object

## 2) *Partial* schema inference

Note that it is possible to specify schema for only a *subset* of columns. (This is okay for interactive usage, but should not be done in production):

In [8]:
%%time
schema_pd = {'id': 'string'}

df = pd.read_csv(
    'data/missing_as_empty_string.csv',
    engine='pyarrow',
    dtype=schema_pd,
)
df.dtypes

CPU times: user 1.26 s, sys: 453 ms, total: 1.71 s
Wall time: 533 ms


id                    string[python]
vendor_id                    float64
pickup_datetime        datetime64[s]
dropoff_datetime       datetime64[s]
passenger_count              float64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag            object
trip_duration                float64
dtype: object

In this case, it works to specify that vendor ID is an integer, even though it is encoded like a float (e.g., "2.0" rather than "2").

In [9]:
%%time
schema_pd = {
    'vendor_id': 'int16[pyarrow]',
}
df = pd.read_csv(
    'data/missing_as_empty_string.csv',
    engine='pyarrow',
    dtype=schema_pd,
)
df.dtypes

CPU times: user 1.43 s, sys: 263 ms, total: 1.69 s
Wall time: 506 ms


id                            object
vendor_id             int16[pyarrow]
pickup_datetime        datetime64[s]
dropoff_datetime       datetime64[s]
passenger_count              float64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag            object
trip_duration                float64
dtype: object

In [10]:
df.store_and_fwd_flag.unique()

array(['N', None, 'Y'], dtype=object)

## 3) Reading using *explicit* schema
Again, this works for reasonably-standard encodings of missing values (in our case, "", "NA", and a combination of "NA" and "nan":
### Works: Missing values as empty strings

In [11]:
%%time
schema_pd = {
    'id': 'string[pyarrow]',
    'vendor_id': 'int16[pyarrow]',
    'pickup_datetime': 'timestamp[s][pyarrow]',
    'dropoff_datetime': 'timestamp[s][pyarrow]',
    'passenger_count': 'int8[pyarrow]',
    'pickup_longitude': 'float32[pyarrow]',
    'pickup_latitude': 'float32[pyarrow]',
    'dropoff_longitude': 'float32[pyarrow]',
    'dropoff_latitude': 'float32[pyarrow]',
    'store_and_fwd_flag': 'string[pyarrow]',
    'trip_duration': 'float32[pyarrow]',
}
df = pd.read_csv(
    'data/missing_as_empty_string.csv',
    engine='pyarrow',
    dtype=schema_pd,
)
dtypes_schema = df.dtypes
dtypes_schema

CPU times: user 1.56 s, sys: 278 ms, total: 1.83 s
Wall time: 592 ms


id                          string[pyarrow]
vendor_id                    int16[pyarrow]
pickup_datetime       timestamp[s][pyarrow]
dropoff_datetime      timestamp[s][pyarrow]
passenger_count               int8[pyarrow]
pickup_longitude             float[pyarrow]
pickup_latitude              float[pyarrow]
dropoff_longitude            float[pyarrow]
dropoff_latitude             float[pyarrow]
store_and_fwd_flag          string[pyarrow]
trip_duration                float[pyarrow]
dtype: object

### Works: Missing encoded as 'NA'

In [36]:
%%time
df = pd.read_csv(
    PATH_MISSING_AS_STRING,
    engine='pyarrow',
    dtype_backend='pyarrow',
    dtype=schema_pd,
)

df.dtypes

CPU times: user 1.01 s, sys: 1.06 s, total: 2.07 s
Wall time: 385 ms


id                          string[pyarrow]
vendor_id                    int16[pyarrow]
pickup_datetime       timestamp[s][pyarrow]
dropoff_datetime      timestamp[s][pyarrow]
passenger_count               int8[pyarrow]
pickup_longitude             float[pyarrow]
pickup_latitude              float[pyarrow]
dropoff_longitude            float[pyarrow]
dropoff_latitude             float[pyarrow]
store_and_fwd_flag          string[pyarrow]
trip_duration                float[pyarrow]
dtype: object

### Works: Missing encoded as 'NA' or 'nan'

In [57]:
%%time
df = pd.read_csv(
    PATH_MISSING_AS_MULTIPLE_STRINGS,
    engine='pyarrow',
    dtype_backend='pyarrow',
    dtype=schema_pd,
)

df.dtypes

CPU times: user 1.14 s, sys: 609 ms, total: 1.75 s
Wall time: 502 ms


id                          string[pyarrow]
vendor_id                    int16[pyarrow]
pickup_datetime       timestamp[s][pyarrow]
dropoff_datetime      timestamp[s][pyarrow]
passenger_count               int8[pyarrow]
pickup_longitude             float[pyarrow]
pickup_latitude              float[pyarrow]
dropoff_longitude            float[pyarrow]
dropoff_latitude             float[pyarrow]
store_and_fwd_flag          string[pyarrow]
trip_duration                float[pyarrow]
dtype: object

### Fails: Missing values encoded as "???"
However, reading the file fails with less-standard encodings of missing values, such as "???".

In [50]:
try:
    pd.read_csv(
        MISSING_AS_QUESTION_MARKS,
        engine='pyarrow',
        dtype_backend='pyarrow',
        dtype=schema_pd,
    )
except pa.ArrowInvalid as e:
    print(e)

Failed to parse string: '1.0' as a scalar of type int16: Error while type casting for column 'vendor_id'



Interestingly, **the error message is not always very useful in telling us why exactly the data could not be parsed into the supplied schema:**
In the below case, it complains that the string "1.0" could not be parsed to integer. This makes it sound like it is some sort of limitation on the side of Arrow not being able to perform the simple casting. However, we know from the above examples that this is not in fact the problem, because the different encoding of missing values vendor_id was successfully parsed to an individual column without any issues. The actual problem is the string "???" which it cannot cast into an integer (or recognize as a missing value indicator). Presumably the error message just gives us the value in the first row of the column failing, rather than the actual element in that column that is causing the issue. 

Let's remove vendor_id from the schema and see what error the next column raises:

In [52]:
try:
    pd.read_csv(
        MISSING_AS_QUESTION_MARKS,
        engine='pyarrow',
        dtype_backend='pyarrow',
        dtype={
            'id': 'string[pyarrow]',
            # 'vendor_id': 'int16[pyarrow]',
            'pickup_datetime': 'timestamp[s][pyarrow]',
            'dropoff_datetime': 'timestamp[s][pyarrow]',
            'passenger_count': 'int8[pyarrow]',
            'pickup_longitude': 'float32[pyarrow]',
            'pickup_latitude': 'float32[pyarrow]',
            'dropoff_longitude': 'float32[pyarrow]',
            'dropoff_latitude': 'float32[pyarrow]',
            'store_and_fwd_flag': 'string[pyarrow]',
            'trip_duration': 'float32[pyarrow]',
        },
    )
except pa.ArrowInvalid as e:
    print(e)

Failed to parse string: '???' as a scalar of type timestamp[s]: Error while type casting for column 'pickup_datetime'


Interestingly, in the case of the failed parsing of the timestamp, it does signal out "???" as the cause of the problem.

Let's try removing the timestamp columns from the schema as well:

In [48]:
try:
    pd.read_csv(
        MISSING_AS_QUESTION_MARKS,
        engine='pyarrow',
        dtype_backend='pyarrow',
        dtype={
            'id': 'string[pyarrow]',
            # 'vendor_id': 'int16[pyarrow]',
            # 'pickup_datetime': 'timestamp[s][pyarrow]',
            # 'dropoff_datetime': 'timestamp[s][pyarrow]',
            'passenger_count': 'int8[pyarrow]',
            'pickup_longitude': 'float32[pyarrow]',
            'pickup_latitude': 'float32[pyarrow]',
            'dropoff_longitude': 'float32[pyarrow]',
            'dropoff_latitude': 'float32[pyarrow]',
            'store_and_fwd_flag': 'string[pyarrow]',
            'trip_duration': 'float32[pyarrow]',
        },
    )
except pa.ArrowInvalid as e:
    print(e)

Failed to parse string: '1.0' as a scalar of type int8: Error while type casting for column 'passenger_count'


The first column causing problems in the schema is an integer column, and we get the same result as for the previous integer column, vendor_id: It misleadingly complains that the string of "1.0" cannot be parsed to integer.

Let's try one more thing, namely what happens in the case of floats:

In [47]:
try:
    pd.read_csv(
        MISSING_AS_QUESTION_MARKS,
        engine='pyarrow',
        dtype_backend='pyarrow',
        dtype={
            'id': 'string[pyarrow]',
            # 'vendor_id': 'int16[pyarrow]',
            # 'pickup_datetime': 'timestamp[s][pyarrow]',
            # 'dropoff_datetime': 'timestamp[s][pyarrow]',
            # 'passenger_count': 'int8[pyarrow]',
            'pickup_longitude': 'float32[pyarrow]',
            'pickup_latitude': 'float32[pyarrow]',
            'dropoff_longitude': 'float32[pyarrow]',
            'dropoff_latitude': 'float32[pyarrow]',
            'store_and_fwd_flag': 'string[pyarrow]',
            'trip_duration': 'float32[pyarrow]',
        },
    )
except pa.ArrowInvalid as e:
    print(e)

Failed to parse string: '???' as a scalar of type float: Error while type casting for column 'pickup_longitude'


Interestingly, it does give us the right error message for floats, like in the case of timestamps. So **the problem seems to be confined to integers.**